**PACCAR Data validation assignment**

This ipynb notebook contains the solution to the data validation assignment.

I have explained the steps involved in successive code sections.


**Reading input files and consolidating**

In the first cell I have read the csv and json files into 2 dataframes. In both dataframes I have excluded entries where there are no runs and wickets for a player.

In [ ]:
import pandas as pd

#Reading entries from csv file
from_csv = pd.read_csv('PACCAR/input/testDataSet1.csv', sep=';')

#Dropping entries where player has no runs and wickets
from_csv = from_csv.dropna(subset=['runs','wickets'],how='all')
from_csv = from_csv.reset_index(drop='True')

#Reading entries from json file
from_json = pd.read_json('PACCAR/input/testDataSet2.json',lines='True')

#Dropping entries where player has no runs and wickets
from_json = from_json.dropna(subset=['runs','wickets'],how='all')
from_json = from_json.reset_index(drop='True')

#Consolidating both entries from both dataframes
consol_df = pd.concat([from_csv,from_json],ignore_index=True)


**Data Processor Logic**

Applying 'Data Processor' logic desribed in the problem statement to the consolidated data frame to get valid entries. An additional column to store 'playerType' has been accordingly added to the data frame.

In [ ]:

#Filter players based on age
consol_df = consol_df[(consol_df["age"] >= 15) & (consol_df["age"] <=50)]
consol_df= consol_df.reset_index(drop='True')

#Add new column for Player Type based on conditions
consol_df.loc[ ( consol_df["runs"] > 500 ) & ( consol_df["wickets"] > 50 ) , "playerType" ] = "All-Rounder"

consol_df.loc[ ( consol_df["runs"] > 500 ) & ( consol_df["wickets"] < 50 ) , "playerType" ] = "Batsman"

consol_df.loc[ ( consol_df["runs"] < 500 ) , "playerType" ] = "Bowler"

consol_df = consol_df.sort_values(by=['eventType'],ignore_index=True)

print("Dataframe containing valid entries")
print(consol_df.head(20))


Dataframe containing valid entries
  eventType playerName  age    runs  wickets   playerType
0       ODI     ABC123   25  1000.0    171.0  All-Rounder
1       ODI     EFG789   24  2500.0     10.0      Batsman
2       ODI     WSY128   35    10.0     15.0       Bowler
3       ODI     abc456   34    10.0      0.0       Bowler
4       ODI    bnhg678   45  4000.0    200.0  All-Rounder
5      TEST     CDE456   45   100.0    100.0       Bowler
6      TEST      QRS67   24   100.0    100.0       Bowler
7      TEST     abc456   34  1000.0    600.0  All-Rounder
8      TEST    bnhg678   45   100.0    600.0       Bowler


**Reading Output files generated by Data Processor to validate results**

In this cell, I have read the Output files into 2 data frames and processed them to compare the entries in the repective files with the consolidated data frame containing valid entries from the previous cell.

In case of invalid entries, the column "Results" has been populated with FAIL and for valid entries as PASS.

Towards the end I have displayed the entries in the test_reults.csv file.

In [ ]:
#Read o/p files of data processor into seaprate dataframes

odi_opfile = pd.read_csv('PACCAR/output/odi.csv', sep=';')
test_opfile = pd.read_csv('PACCAR/output/test.csv', sep=';')

#Compare entries from the output files with the dataframe containing valid entries and mark
#them accordingly

odi_op_merg = odi_opfile.merge(consol_df, on=["eventType","playerName"] , how= 'left')
odi_op_merg["Result"] = odi_op_merg.apply( lambda row: "PASS" if row["playerType_x"] == row["playerType_y"] else "FAIL",axis=1)
odi_op_merg.drop(columns=["age_y","runs_y","wickets_y","playerType_y"],inplace=True)
odi_op_merg.rename(columns= {"age_x":"age","runs_x":"runs","wickets_x":"wickets","playerType_x":"playerType"},inplace=True)

print('Test result of odi.csv file')
print(odi_op_merg.head(20))

test_op_merg = test_opfile.merge(consol_df, on=["eventType","playerName"] , how= 'left')
test_op_merg["Result"] = test_op_merg.apply( lambda row: "PASS" if row["playerType_x"] == row["playerType_y"] else "FAIL",axis=1)
test_op_merg.drop(columns=["age_y","runs_y","wickets_y","playerType_y"],inplace=True)
test_op_merg.rename(columns= {"age_x":"age","runs_x":"runs","wickets_x":"wickets","playerType_x":"playerType"},inplace=True)
print()
print('Test result of test.csv file')
print(test_op_merg.head(20))


#Generate test results file consolidating the above 2 dataframes.

test_results = pd.concat([odi_op_merg,test_op_merg],ignore_index=True)

print()
print()
print('Consolidated test results')
print(test_results.head(50))

# #Save the above dataframe as a csv file
test_results.to_csv('PACCAR/test_results/test_results.csv',index=False)



Test result of odi.csv file
  eventType playerName  age  runs  wickets   playerType Result
0       ODI     ABC123   25  1000      171  All-Rounder   PASS
1       ODI     EFG789   24  2500       10      Batsman   PASS
2       ODI     HIJ012   10   250       29       Bowler   FAIL
3       ODI     abc456   34    10        0       Bowler   PASS
4       ODI    bnhg678   45  4000      200  All-Rounder   PASS

Test result of test.csv file
  eventType playerName  age  runs  wickets   playerType Result
0      TEST     CDE456   45   100      100       Bowler   PASS
1      TEST     KLM901   51  4000      800  All-Rounder   FAIL
2      TEST      QRS67   24   100      100       Bowler   PASS
3      TEST     abc456   34  1000      600  All-Rounder   PASS
4      TEST    bnhg678   45   100      600      Batsman   FAIL


Consolidated test results
  eventType playerName  age  runs  wickets   playerType Result
0       ODI     ABC123   25  1000      171  All-Rounder   PASS
1       ODI     EFG789   24  250